## Integration of functions loaded in MPS

This notebook shows how to integrate functions loaded in MPS by contracting them with predefined quadrature MPS. These quadrature MPS can be straightforwardly generalized to multivariate scenarios by means of tensor products in both qubit orders, either serial or interleaved. Moreover, this integration procedure avoids the *curse of dimensionality*, as it has a cost that scales polynomially with the dimension of the function instead than exponentially.

The main prerequisite for this *quantum-inspired* integration method is to dispose of a prior multivariate function loaded in MPS. This can be done following several methods, such as MPS Chebyshev expansions (see the `chebyshev_composition.ipynb` example), tensor cross-interpolation (TCI, see the `tt-cross.ipynb` example), multiscale interpolative constructions, etc.

For this example, we are going to load a trivially simple multivariate function 

$f(x_1, \ldots, x_{10}) = \prod_{i=1}^{10} x_i^3$.

The integral of this function in $\Omega = [-1, 1] \times \ldots \times [-1, 1]$ is naively zero. Even though it can be analytically constructed, we use tensor cross-interpolation as it generalizes to a wider range of functions. Then, we integrate it following a Clenshaw-Curtis quadrature rule.

In [5]:
import numpy as np

from seemps.analysis.mesh import ChebyshevInterval, Mesh
from seemps.analysis.cross import BlackBoxLoadMPS, cross_dmrg
import seemps.tools

seemps.tools.DEBUG = 2

start, stop = -1, 1
num_qubits = 3
interval = ChebyshevInterval(
    start, stop, 2**num_qubits, endpoints=True
)  # Chebyshev extrema

dimension = 10
mesh = Mesh([interval] * dimension)

func = lambda tensor: np.sum(tensor**3, axis=0)  # x^3 * y^3 * ...
black_box = BlackBoxLoadMPS(func, mesh)

mps_func = cross_dmrg(black_box).mps

 Cross sweep   1 with error(1000 samples in norm-inf)=3.2845046914370504, maxbond=2, evals(cumulative)=228
 Cross sweep   1 with error(1000 samples in norm-inf)=2.842170943040401e-14, maxbond=3, evals(cumulative)=816
 State converged within tolerance 1e-12


Next, we construct the quadrature MPS. We can integrate the function directly using the auxiliar routine `integrate_mps`, but in this example we construct the quadrature manually.

In [9]:
from seemps.analysis.integration import mps_clenshaw_curtis
from seemps.analysis.factories import mps_tensor_product

mps_quad_1d = mps_clenshaw_curtis(start, stop, num_qubits)
mps_quad_10d = mps_tensor_product([mps_quad_1d] * dimension)

Finally, integrate the MPS by taking the scalar product `scprod` of the function MPS with the quadrature MPS.

In [10]:
from seemps.state import scprod

integral_exact = 0
integral_mps = scprod(mps_func, mps_quad_10d)

print("Integration error: ", np.max(np.abs(integral_exact - integral_mps)))

Integration error:  4.195754854663392e-12
